# Bellevue University
## Spring 2022
### DSC 650 : Big Data
### Week 7 : Machine Learning Fundamentals

## Assignment 7.1a
04/29/2022
##### By: Yograj Karki

In [6]:
import os
import json
from pathlib import Path
import gzip
import hashlib
import shutil
import pandas as pd
import pygeohash
import s3fs

endpoint_url='https://storage.budsc.midwest-datascience.com'
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')

if results_dir.exists():
    shutil.rmtree(results_dir)
results_dir.mkdir(parents=True, exist_ok=True)

def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]
    return records


AttributeError: partially initialized module 'fsspec' has no attribute 'utils' (most likely due to a circular import)

In [94]:
from pyarrow.json import read_json
import pyarrow.parquet as pq
def create_parquet_dataset():
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    parquet_output_path = results_dir.joinpath('routes.parquet')
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            ## TODO: Use Apache Arrow to create Parquet table and save the dataset
            table = read_json(f)
            print(table)
            pq.write_table(table, parquet_output_path, compression='none')
            

create_parquet_dataset()

pyarrow.Table
airline: struct<airline_id: int64, name: string, alias: string, iata: string, icao: string, callsign: string, country: string, active: bool>
  child 0, airline_id: int64
  child 1, name: string
  child 2, alias: string
  child 3, iata: string
  child 4, icao: string
  child 5, callsign: string
  child 6, country: string
  child 7, active: bool
src_airport: struct<airport_id: int64, name: string, city: string, country: string, iata: string, icao: string, latitude: double, longitude: double, altitude: int64, timezone: double, dst: string, tz_id: string, type: string, source: string>
  child 0, airport_id: int64
  child 1, name: string
  child 2, city: string
  child 3, country: string
  child 4, iata: string
  child 5, icao: string
  child 6, latitude: double
  child 7, longitude: double
  child 8, altitude: int64
  child 9, timezone: double
  child 10, dst: string
  child 11, tz_id: string
  child 12, type: string
  child 13, source: string
dst_airport: struct<airport_id: 

In [95]:
parquet_output_path = results_dir.joinpath('routes.parquet')
pq = pd.read_parquet(parquet_output_path, engine='fastparquet')
print(list(pq.columns.values))

['codeshare', 'equipment', 'airline.airline_id', 'airline.name', 'airline.alias', 'airline.iata', 'airline.icao', 'airline.callsign', 'airline.country', 'airline.active', 'src_airport.airport_id', 'src_airport.name', 'src_airport.city', 'src_airport.country', 'src_airport.iata', 'src_airport.icao', 'src_airport.latitude', 'src_airport.longitude', 'src_airport.altitude', 'src_airport.timezone', 'src_airport.dst', 'src_airport.tz_id', 'src_airport.type', 'src_airport.source', 'dst_airport.airport_id', 'dst_airport.name', 'dst_airport.city', 'dst_airport.country', 'dst_airport.iata', 'dst_airport.icao', 'dst_airport.latitude', 'dst_airport.longitude', 'dst_airport.altitude', 'dst_airport.timezone', 'dst_airport.dst', 'dst_airport.tz_id', 'dst_airport.type', 'dst_airport.source']


In [96]:
partitions = (
        ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
        ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
        ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
        ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
    )

In [97]:
partitions_keys = (
        'A', 'B', 'C-D', 'E-F',
        'G-H', 'I-J', 'K-L', 'M',
        'N', 'O-P', 'Q-R', 'S-T',
        'U', 'V', 'W-X', 'Y-Z'
    )

In [98]:
parts_k_v = dict(zip(partitions_keys, partitions))
print(parts_k_v)

{'A': ('A', 'A'), 'B': ('B', 'B'), 'C-D': ('C', 'D'), 'E-F': ('E', 'F'), 'G-H': ('G', 'H'), 'I-J': ('I', 'J'), 'K-L': ('K', 'L'), 'M': ('M', 'M'), 'N': ('N', 'N'), 'O-P': ('O', 'P'), 'Q-R': ('Q', 'R'), 'S-T': ('S', 'T'), 'U': ('U', 'U'), 'V': ('V', 'V'), 'W-X': ('W', 'X'), 'Y-Z': ('Y', 'Z')}


In [99]:
def get_key(val):
    for key, value in parts_k_v.items():
        if val in value:
            return key
    return "0"

In [100]:
pq['key'] = pq['src_airport.iata'] + pq['dst_airport.iata'] + pq['airline.iata']
pq['partition_value'] = pq['key'].str[:1]
pq['kv_key'] = pq.apply(lambda x: get_key(x.partition_value), axis=1)

In [101]:
# remove invalid keys
pq = pq[pq.kv_key != "0"].astype('float32', errors='ignore')

In [102]:
pq.head()

,codeshare,equipment,airline.airline_id,airline.name,airline.alias,airline.iata,airline.icao,airline.callsign,airline.country,airline.active,...,dst_airport.longitude,dst_airport.altitude,dst_airport.timezone,dst_airport.dst,dst_airport.tz_id,dst_airport.type,dst_airport.source,key,partition_value,kv_key
0,0.0,[CR2],410.0,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,1.0,...,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,AERKZN2B,A,A
1,0.0,[CR2],410.0,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,1.0,...,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,ASFKZN2B,A,A
2,0.0,[CR2],410.0,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,1.0,...,43.081902,1054.0,3.0,N,Europe/Moscow,airport,OurAirports,ASFMRV2B,A,A
3,0.0,[CR2],410.0,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,1.0,...,49.278702,411.0,3.0,N,Europe/Moscow,airport,OurAirports,CEKKZN2B,C,C-D
4,0.0,[CR2],410.0,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,1.0,...,82.650703,365.0,7.0,N,Asia/Krasnoyarsk,airport,OurAirports,CEKOVB2B,C,C-D


In [104]:
import pyarrow as pa
import pyarrow.parquet as parpq
pq_tab = pa.Table.from_pandas(pq)

parpq.write_to_dataset(
    pq_tab,
    root_path=results_dir.joinpath('kv'),
    partition_cols=['kv_key'],
)


# Assignment 7.1.b

In [105]:
import hashlib

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

In [106]:
pq['key'] = pq['src_airport.iata']+pq['dst_airport.iata']+pq['airline.iata']
pq['hashed'] = pq.apply(lambda x: hash_key(x.key), axis=1)
pq['hash_key'] = pq['hashed'].str[:1]

In [107]:
pq_tab1 = pa.Table.from_pandas(pq)

parpq.write_to_dataset(
    pq_tab1,
    root_path=results_dir.joinpath('hash'),
    partition_cols=['hash_key'],
)

# Assignment 7.1.c

In [109]:
#get hash for datacenters
datacenters = {}

datacenters['west'] = pygeohash.encode(45.5945645, -121.1786823)
datacenters['central'] = pygeohash.encode(41.1544433, -96.0422378)
datacenters['east'] = pygeohash.encode(39.08344, -77.6497145)

print(datacenters)

{'west': 'c21g6s0rs4c7', 'central': '9z7dnebnj8kb', 'east': 'dqby34cjw922'}


In [110]:
def closest_datacenter(latitude, longitude):
    geohash = pygeohash.encode(latitude, longitude)
    dist_dict = {}
    closest_datacenter = ''
    last_distance = None
    for key, value in datacenters.items():
        dist = pygeohash.geohash_approximate_distance(str(geohash), str(value))
        dist_dict[key] = dist
        if (last_distance == None) or (dist < last_distance):
            closest_datacenter = key
            last_distance = dist
        
    return closest_datacenter

In [113]:
pq['datacenter'] = pq[['src_airport.latitude', 'src_airport.longitude']].apply(lambda x: closest_datacenter(x[0], x[1]), axis=1)

In [114]:
pq_tab2 = pa.Table.from_pandas(pq)

parpq.write_to_dataset(
    pq_tab2,
    root_path=results_dir.joinpath('geo'),
    partition_cols=['datacenter'],
)

# Assignment 7.1.d

In [128]:
pq['airline.airline_id'].head()

0    410.0
1    410.0
2    410.0
3    410.0
4    410.0
Name: airline.airline_id, dtype: float32

In [133]:
def balance_partitions(keys, num_partitions):
    ac = keys.cumsum()

    #sum of the entire array
    partsum = ac[-1]//num_partitions 

    #generates the cumulative sums of each part
    cum_part_sums = np.array(range(1,p))*partsum

    #finds the indices 
    inds = np.searchsorted(ac,cum_part_sums) 

    #split into approximately equal-sum arrays
    parts = np.split(arr,inds)

    return parts

In [134]:
keys = list(pq['airline.airline_id'])
num_partitions=7

In [135]:
print(balance_partitions(keys, num_partitions))

[{-1.0: 1}, {10.0: 1}, {21.0: 1}, {24.0: 1}, {28.0: 1}, {29.0: 1}, {32.0: 1}, {35.0: 1}, {42.0: 1}, {43.0: 1}, {55.0: 1}, {68.0: 1}, {83.0: 1}, {90.0: 1}, {96.0: 1}, {106.0: 1}, {109.0: 1}, {116.0: 1}, {125.0: 1}, {130.0: 1}, {132.0: 1}, {137.0: 1}, {139.0: 1}, {146.0: 1}, {153.0: 1}, {179.0: 1}, {197.0: 1}, {214.0: 1}, {218.0: 1}, {220.0: 1}, {221.0: 1}, {225.0: 1}, {231.0: 1}, {240.0: 1}, {241.0: 1}, {242.0: 1}, {246.0: 1}, {312.0: 1}, {316.0: 1}, {319.0: 1}, {321.0: 1}, {324.0: 1}, {328.0: 1}, {329.0: 1}, {330.0: 1}, {333.0: 1}, {336.0: 1}, {338.0: 1}, {341.0: 1}, {345.0: 1}, {386.0: 1}, {397.0: 1}, {410.0: 1}, {412.0: 1}, {426.0: 1}, {439.0: 1}, {442.0: 1}, {462.0: 1}, {470.0: 1}, {476.0: 1}, {477.0: 1}, {491.0: 1}, {502.0: 1}, {503.0: 1}, {508.0: 1}, {515.0: 1}, {524.0: 1}, {543.0: 1}, {563.0: 1}, {567.0: 1}, {569.0: 1}, {576.0: 1}, {595.0: 1}, {596.0: 1}, {603.0: 1}, {608.0: 1}, {622.0: 1}, {641.0: 1}, {683.0: 1}, {690.0: 2}, {692.0: 2}, {751.0: 2}, {753.0: 2}, {794.0: 2}, {807.0